<a href="https://colab.research.google.com/github/ZeusKane/serigne-kane/blob/serigne-kane/Exercice_1_corrig%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Introduction to Machine Learning – Titanic Dataset (Version Corrigée)
# Ce notebook introduit l'apprentissage supervisé avec preprocessing, pipelines et évaluation

# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

print("🚢 Titanic Machine Learning Pipeline - Version Complète")
print("=" * 60)

# 📥 Chargement du Dataset Titanic
print("\n📥 Chargement du Dataset Titanic...")
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
print(f"Forme du dataset: {df.shape}")
print("\nPremières 5 lignes:")
print(df.head())

print("\nInformations sur le dataset:")
print(df.info())
print("\nValeurs manquantes par colonne:")
print(df.isnull().sum())

# 🧹 Sélection des Features et Target
print("\n🧹 Sélection des Features et Target...")
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
target = 'Survived'
X = df[features]
y = df[target]
print(f"Features sélectionnées: {features}")
print(f"Variable cible: {target}")
print(f"Forme X: {X.shape}, Forme y: {y.shape}")

# 🔧 Définition du Pipeline de Preprocessing
print("\n🔧 Configuration du Pipeline de Preprocessing...")

# Features numériques
numeric_features = ['Age', 'Fare']
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Features catégorielles
categorical_features = ['Pclass', 'Sex', 'Embarked']
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combinaison des transformateurs
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

print("Pipeline de preprocessing configuré avec succès!")

# 🔁 Pipeline Complet avec Régression Logistique
print("\n🔁 Création du Pipeline Complet avec Régression Logistique...")
clf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Division des données
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Taille du set d'entraînement: {X_train.shape[0]}")
print(f"Taille du set de test: {X_test.shape[0]}")

# Entraînement du modèle
print("\n🎯 Entraînement du modèle Régression Logistique...")
clf_pipeline.fit(X_train, y_train)

# Évaluation
y_pred = clf_pipeline.predict(X_test)
print("\n📊 Résultats Régression Logistique:")
print(classification_report(y_test, y_pred))
lr_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {lr_accuracy:.4f}")

# Sauvegarde du pipeline entraîné
print("\n💾 Sauvegarde du pipeline entraîné...")
joblib.dump(clf_pipeline, "titanic_pipeline.pkl")
print("✅ Pipeline sauvegardé sous 'titanic_pipeline.pkl'")

print("\n" + "=" * 60)
print("EXERCICE 1: Essayer un Classificateur Différent")
print("=" * 60)

# Exercice 1: Random Forest Classifier
print("\n🌲 Entraînement avec Random Forest Classifier...")
rf_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

print("\n📊 Résultats Random Forest:")
print(classification_report(y_test, y_pred_rf))
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {rf_accuracy:.4f}")

print(f"\n🔍 Comparaison des modèles:")
print(f"Régression Logistique Accuracy: {lr_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"Amélioration: {rf_accuracy - lr_accuracy:.4f}")

# Analyse des features importantes (Random Forest)
feature_names = (numeric_features +
                list(rf_pipeline.named_steps['preprocessing']
                    .named_transformers_['cat']
                    .named_steps['encoder']
                    .get_feature_names_out(categorical_features)))
feature_importance = rf_pipeline.named_steps['classifier'].feature_importances_

print(f"\n🎯 Importance des features (Random Forest):")
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)
print(feature_importance_df.head(10))

print("\n" + "=" * 60)
print("EXERCICE 2: Utiliser la Validation Croisée")
print("=" * 60)

# Exercice 2: Cross-Validation
print("\n🔄 Validation croisée à 5 plis...")

# Validation croisée pour la Régression Logistique
cv_scores_lr = cross_val_score(clf_pipeline, X, y, cv=5, scoring='accuracy')
print(f"\nScores CV Régression Logistique: {cv_scores_lr.round(4)}")
print(f"Accuracy CV moyenne: {cv_scores_lr.mean():.4f} (+/- {cv_scores_lr.std() * 2:.4f})")

# Validation croisée pour Random Forest
cv_scores_rf = cross_val_score(rf_pipeline, X, y, cv=5, scoring='accuracy')
print(f"\nScores CV Random Forest: {cv_scores_rf.round(4)}")
print(f"Accuracy CV moyenne: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std() * 2:.4f})")

print(f"\n🎯 Analyse de la stabilité du modèle:")
print(f"Écart-type Régression Logistique: {cv_scores_lr.std():.4f}")
print(f"Écart-type Random Forest: {cv_scores_rf.std():.4f}")
print("Un écart-type plus faible indique un modèle plus stable entre les plis.")

print("\n" + "=" * 60)
print("EXERCICE 3: Ingénierie de Features")
print("=" * 60)

# Exercice 3: Feature Engineering
print("\n🔧 Ajout de la feature FamilySize...")
df['FamilySize'] = df['SibSp'] + df['Parch']
print(f"Statistiques FamilySize:")
print(df['FamilySize'].describe())

# Créer des features additionnelles
df['IsAlone'] = (df['FamilySize'] == 0).astype(int)
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# Regrouper les titres rares
df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                  'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

print(f"\nDistribution des titres:")
print(df['Title'].value_counts())

# Features améliorées
features_enhanced = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone', 'Title']
X_enhanced = df[features_enhanced]

# Pipeline de preprocessing amélioré
numeric_features_enhanced = ['Age', 'Fare', 'FamilySize']
categorical_features_enhanced = ['Pclass', 'Sex', 'Embarked', 'IsAlone', 'Title']

numeric_transformer_enhanced = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_enhanced = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_enhanced = ColumnTransformer([
    ('num', numeric_transformer_enhanced, numeric_features_enhanced),
    ('cat', categorical_transformer_enhanced, categorical_features_enhanced)
])

# Pipeline amélioré
enhanced_pipeline = Pipeline([
    ('preprocessing', preprocessor_enhanced),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entraînement et évaluation du modèle amélioré
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(
    X_enhanced, y, test_size=0.2, random_state=42, stratify=y
)

enhanced_pipeline.fit(X_train_enh, y_train_enh)
y_pred_enh = enhanced_pipeline.predict(X_test_enh)

print("\n📊 Résultats du Modèle Amélioré (avec nouvelles features):")
print(classification_report(y_test_enh, y_pred_enh))
enhanced_accuracy = accuracy_score(y_test_enh, y_pred_enh)
print(f"Accuracy du Modèle Amélioré: {enhanced_accuracy:.4f}")

print(f"\n🔍 Impact de l'ingénierie de features:")
print(f"Random Forest Original:    {rf_accuracy:.4f}")
print(f"Modèle Amélioré:          {enhanced_accuracy:.4f}")
print(f"Amélioration:             {enhanced_accuracy - rf_accuracy:.4f}")

# Validation croisée pour le modèle amélioré
cv_scores_enhanced = cross_val_score(enhanced_pipeline, X_enhanced, y, cv=5, scoring='accuracy')
print(f"\nCV Modèle Amélioré: {cv_scores_enhanced.mean():.4f} (+/- {cv_scores_enhanced.std() * 2:.4f})")

# Sauvegarde du pipeline amélioré
joblib.dump(enhanced_pipeline, "titanic_enhanced_pipeline.pkl")
print("✅ Pipeline amélioré sauvegardé sous 'titanic_enhanced_pipeline.pkl'")

print("\n" + "=" * 60)
print("EXERCICE 4: Interface Streamlit")
print("=" * 60)

# Code Streamlit pour l'interface
streamlit_code = '''
# streamlit_titanic_app.py
import streamlit as st
import joblib
import pandas as pd
import numpy as np

# Charger le modèle entraîné
@st.cache_resource
def load_model():
    try:
        return joblib.load("titanic_enhanced_pipeline.pkl")
    except FileNotFoundError:
        st.error("Modèle non trouvé. Assurez-vous que 'titanic_enhanced_pipeline.pkl' existe.")
        return None

def main():
    st.set_page_config(page_title="Titanic Survival Predictor", page_icon="🚢")

    st.title("🚢 Prédicteur de Survie du Titanic")
    st.write("Prédisez la survie d'un passager du Titanic basé sur ses caractéristiques.")

    # Charger le modèle
    model = load_model()
    if model is None:
        return

    # Interface utilisateur
    st.sidebar.header("Informations du Passager")

    # Champs d'entrée
    pclass = st.sidebar.selectbox("Classe du Passager", [1, 2, 3],
                                 help="1 = Première Classe, 2 = Deuxième Classe, 3 = Troisième Classe")
    sex = st.sidebar.selectbox("Sexe", ["male", "female"])
    age = st.sidebar.slider("Âge", 0, 100, 30, help="Âge en années")
    fare = st.sidebar.slider("Prix du billet", 0.0, 500.0, 32.0, step=0.1,
                            help="Prix du billet en livres")
    embarked = st.sidebar.selectbox("Port d'embarquement", ["S", "C", "Q"],
                                   help="S = Southampton, C = Cherbourg, Q = Queenstown")

    # Informations familiales
    st.sidebar.subheader("Informations Familiales")
    sibsp = st.sidebar.number_input("Frères/Sœurs/Conjoints à bord", 0, 8, 1)
    parch = st.sidebar.number_input("Parents/Enfants à bord", 0, 6, 0)
    family_size = sibsp + parch
    is_alone = 1 if family_size == 0 else 0

    # Titre (simplifié pour l'interface)
    title_options = ["Mr", "Mrs", "Miss", "Master", "Rare"]
    title = st.sidebar.selectbox("Titre", title_options)

    st.sidebar.write(f"**Taille de famille: {family_size}**")
    st.sidebar.write(f"**Voyage seul: {'Oui' if is_alone else 'Non'}**")

    # Résumé du passager
    st.subheader("Résumé du Passager")
    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric("Classe", pclass)
        st.metric("Sexe", sex)
        st.metric("Âge", f"{age} ans")

    with col2:
        st.metric("Prix", f"£{fare:.2f}")
        st.metric("Embarquement", embarked)
        st.metric("Titre", title)

    with col3:
        st.metric("Taille famille", family_size)
        st.metric("Voyage seul", "Oui" if is_alone else "Non")

    # Prédiction
    if st.button("🎯 Prédire la Survie", type="primary"):
        # Créer DataFrame pour la prédiction
        X_new = pd.DataFrame([[pclass, sex, age, fare, embarked, family_size, is_alone, title]],
                           columns=["Pclass", "Sex", "Age", "Fare", "Embarked",
                                   "FamilySize", "IsAlone", "Title"])

        try:
            # Faire la prédiction
            prediction = model.predict(X_new)[0]
            probability = model.predict_proba(X_new)[0]

            # Afficher les résultats
            st.subheader("Résultats de la Prédiction")

            if prediction == 1:
                st.success("🎉 **SURVÉCU** - Ce passager aurait probablement survécu !")
                st.balloons()
            else:
                st.error("💔 **N'A PAS SURVÉCU** - Ce passager n'aurait probablement pas survécu.")

            # Probabilités
            st.subheader("Détail des Probabilités")
            col1, col2 = st.columns(2)
            with col1:
                st.metric("Probabilité de Survie", f"{probability[1]:.1%}")
            with col2:
                st.metric("Probabilité de Décès", f"{probability[0]:.1%}")

            # Barre de progression
            st.progress(probability[1])

            # Graphique des probabilités
            prob_data = pd.DataFrame({
                'Résultat': ['Décès', 'Survie'],
                'Probabilité': [probability[0], probability[1]]
            })
            st.bar_chart(prob_data.set_index('Résultat'))

        except Exception as e:
            st.error(f"Erreur lors de la prédiction: {str(e)}")

    # Informations sur le modèle
    with st.expander("ℹ️ Informations sur le Modèle"):
        st.write("""
        Ce modèle utilise un Random Forest avec les caractéristiques suivantes:
        - **Features**: Classe, Sexe, Âge, Prix, Port d'embarquement, Taille famille, Voyage seul, Titre
        - **Algorithme**: Random Forest Classifier
        - **Preprocessing**: Imputation des valeurs manquantes, standardisation, encodage one-hot
        - **Performance**: ~82% d'accuracy en validation croisée
        """)

if __name__ == "__main__":
    main()
'''

print("📝 Code de l'application Streamlit généré!")
print("\nPour lancer l'application Streamlit:")
print("1. Sauvegardez le code ci-dessus sous 'streamlit_titanic_app.py'")
print("2. Installez streamlit: pip install streamlit")
print("3. Lancez: streamlit run streamlit_titanic_app.py")

# Sauvegarder le code Streamlit
with open("streamlit_titanic_app.py", "w", encoding='utf-8') as f:
    f.write(streamlit_code)
print("✅ Application Streamlit sauvegardée sous 'streamlit_titanic_app.py'")

print("\n" + "=" * 60)
print("🎯 RÉSUMÉ DES RÉSULTATS")
print("=" * 60)

print(f"\n📊 Comparaison des Performances des Modèles:")
print(f"1. Régression Logistique:        {lr_accuracy:.4f}")
print(f"2. Random Forest:                {rf_accuracy:.4f}")
print(f"3. Modèle Amélioré (RF + feat.): {enhanced_accuracy:.4f}")

print(f"\n🔄 Résultats de la Validation Croisée:")
print(f"• Régression Logistique CV: {cv_scores_lr.mean():.4f} ± {cv_scores_lr.std():.4f}")
print(f"• Random Forest CV:         {cv_scores_rf.mean():.4f} ± {cv_scores_rf.std():.4f}")
print(f"• Modèle Amélioré CV:       {cv_scores_enhanced.mean():.4f} ± {cv_scores_enhanced.std():.4f}")

print(f"\n🚀 Insights Clés:")
print("• Random Forest surpasse généralement la Régression Logistique")
print("• L'ingénierie de features (FamilySize, IsAlone, Title) améliore les performances")
print("• La validation croisée montre une bonne stabilité des modèles")
print("• L'application Streamlit est prête pour des prédictions interactives")

print(f"\n📁 Fichiers Créés:")
print("• titanic_pipeline.pkl (Modèle Régression Logistique)")
print("• titanic_enhanced_pipeline.pkl (Modèle Random Forest Amélioré)")
print("• streamlit_titanic_app.py (Application web interactive)")

print(f"\n🎉 Tous les exercices terminés avec succès!")
print("🚀 Vous pouvez maintenant lancer l'application Streamlit pour tester les prédictions!")

🚢 Titanic Machine Learning Pipeline - Version Complète

📥 Chargement du Dataset Titanic...
Forme du dataset: (891, 12)

Premières 5 lignes:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 31